In [1]:
#COLORACTEL CANCER ABNORMAL TISSUE CLASSIFICATION using 2016 Kather et al.
import tensorflow.keras
import pdb
from tensorflow.keras.layers import BatchNormalization
import numpy as np
import math
from sklearn.model_selection import train_test_split, StratifiedKFold
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input,Concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
#from tensorflow.keras.utils import np_utils
from tensorflow.keras.layers import LeakyReLU
%matplotlib inline
tensorflow.keras.backend.set_image_data_format('channels_last')


In [2]:
# Create CNN model
model = Sequential()
model.add(Conv2D(16, (3, 3),input_shape=(224,224,3),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2, 2)))
          
model.add(Conv2D(16, (3, 3),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
          
model.add(Conv2D(32, (3, 3),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
          
model.add(Conv2D(32, (3, 3),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

model.add(Conv2D(64, (3, 3),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
          
model.add(Conv2D(64, (3, 3),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))

model.add(Conv2D(128, (3, 3),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
          
model.add(Conv2D(128, (3, 3),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
   
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(8, activation='softmax'))

In [3]:
from tensorflow.keras import optimizers
model.compile(optimizer=optimizers.Adam(lr=1e-3), loss='categorical_crossentropy', metrics=['accuracy']) 

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


In [4]:
# Data augmentation
batch_size=4

train_datagen = ImageDataGenerator(rescale=1./255,
                         horizontal_flip = True,
                         vertical_flip = True,
                         width_shift_range = 0.2,
                         height_shift_range = 0.2,
                         zoom_range = 0.2,
                         rotation_range = 180,
                         shear_range=0.2
                        )

valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory('data01/train', 
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)

valid_generator = valid_datagen.flow_from_directory('data01/valid', 
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=True)


Found 3496 images belonging to 8 classes.
Found 752 images belonging to 8 classes.


In [5]:
#Callbacks
def get_callbacks(name_weights, patience_lr):
    mcp_save = ModelCheckpoint(name_weights, save_best_only=True, monitor='val_loss', mode='min')
    reduce_lr_loss = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=patience_lr, verbose=2, min_delta=1e-4, mode='min')
    early_stop_cr=EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10, verbose=2, mode='auto', restore_best_weights=True)
    return [mcp_save, reduce_lr_loss, early_stop_cr]


In [6]:
step_size_train=train_generator.n//train_generator.batch_size
step_size_valid=valid_generator.n//valid_generator.batch_size

name_weights = "conv8L_CRC_13jan22.h5"
callbacks = get_callbacks(name_weights = name_weights, patience_lr=10)

model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   validation_steps=step_size_valid,
                   validation_data=valid_generator,
                   epochs=200,
                   verbose=1,
                   callbacks = callbacks)

C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/200
874/874 [==============================] - 166s 180ms/step - loss: 1.3616 - accuracy: 0.3776 - val_loss: 1.3723 - val_accuracy: 0.4189
Epoch 2/200
874/874 [==============================] - 72s 82ms/step - loss: 1.0590 - accuracy: 0.5200 - val_loss: 0.9109 - val_accuracy: 0.6011
Epoch 3/200
874/874 [==============================] - 71s 81ms/step - loss: 0.9702 - accuracy: 0.5752 - val_loss: 0.9341 - val_accuracy: 0.6968
Epoch 4/200
874/874 [==============================] - 71s 82ms/step - loss: 0.8629 - accuracy: 0.6384 - val_loss: 0.7500 - val_accuracy: 0.7713
Epoch 5/200
874/874 [==============================] - 72s 82ms/step - loss: 0.8052 - accuracy: 0.6739 - val_loss: 0.7291 - val_accuracy: 0.6809
Epoch 6/200
874/874 [==============================] - 71s 82ms/step - loss: 0.7808 - accuracy: 0.6919 - val_loss: 0.6521 - val_accuracy: 0.7420
Epoch 7/200
874/874 [==============================] - 71s 82ms/step - loss: 0.7302 - accuracy: 0.7225 - val_loss: 0.6744 - val_

In [7]:
# Validation accuracy, validation data confusion matrix, area under the roc score
valid_generator = valid_datagen.flow_from_directory('data01/valid', 
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 shuffle=False)

# Ref: https://gist.github.com/RyanAkilos/3808c17f79e77c4117de35aa68447045 accessed on 5 Feb 2020
from sklearn.metrics import precision_score, recall_score, confusion_matrix, accuracy_score, roc_auc_score

#https://github.com/bhattbhavesh91/imbalance_class_sklearn/blob/master/imbalance_class_undersampling_oversampling.ipynb

Y_pred = model.predict_generator(valid_generator)
y_pred=np.argmax(Y_pred,axis=1)

ras=roc_auc_score(valid_generator.classes, Y_pred,multi_class='ovr')
print('Confusion Matrix:')
print(confusion_matrix(valid_generator.classes, y_pred))
print('Accuracy:', accuracy_score(valid_generator.classes, y_pred))
print('One versus rest AUC score:', ras)

Found 752 images belonging to 8 classes.


C:\Users\tiryakiv\miniconda3\envs\tf-latest\lib\site-packages\tensorflow\python\keras\engine\training.py:2001: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


Confusion Matrix:
[[92  1  0  0  0  1  0  0]
 [ 0 76 13  0  5  0  0  0]
 [ 3  6 82  3  0  0  0  0]
 [ 0  0  8 86  0  0  0  0]
 [ 1 23  6  0 55  8  1  0]
 [ 2  0  0  2  3 87  0  0]
 [ 0  0  0  0  1  0 87  6]
 [ 0  0  0  0  0  0  1 93]]
Accuracy: 0.875
One versus rest AUC score: 0.9886745780249628
